<a href="https://www.kaggle.com/code/gamalelansary/ambulance-detector?scriptVersionId=280090580" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [9]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import keras
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy("mixed_float16")
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ambulance-dataset/carsdataset/ambulance/WVS3T9217C81.jpg
/kaggle/input/ambulance-dataset/carsdataset/ambulance/051540V1DZV0.jpg
/kaggle/input/ambulance-dataset/carsdataset/ambulance/GVKGCL0RHNXF.jpg
/kaggle/input/ambulance-dataset/carsdataset/ambulance/47MJULMBMSX5.jpg
/kaggle/input/ambulance-dataset/carsdataset/ambulance/WHN9ELO942KP.jpg
/kaggle/input/ambulance-dataset/carsdataset/ambulance/0178DFJL75X3.jpg
/kaggle/input/ambulance-dataset/carsdataset/ambulance/6L43R3VEMF2A.jpg
/kaggle/input/ambulance-dataset/carsdataset/ambulance/EPHAKGXKMNDV.jpg
/kaggle/input/ambulance-dataset/carsdataset/ambulance/3S2ZR4VBTVZG.jpg
/kaggle/input/ambulance-dataset/carsdataset/ambulance/MTY60OJC2O6E.jpg
/kaggle/input/ambulance-dataset/carsdataset/ambulance/AEW14C929FM1.jpg
/kaggle/input/ambulance-dataset/carsdataset/ambulance/73JPNHB5Y76L.jpg
/kaggle/input/ambulance-dataset/carsdataset/ambulance/51IZSX32MCG0.jpg
/kaggle/input/ambulance-dataset/carsdataset/ambulance/BE6GYI2VUZG0.jpg
/kaggl

In [17]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    '/kaggle/input/ambulance-dataset/carsdataset',
    labels='inferred',
    label_mode='categorical',
    validation_split=0.2,
    subset='training',        
    class_names=None,
    color_mode='rgb',
    batch_size=64,
    image_size=(256, 256),
    shuffle=True,
    seed=42
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    '/kaggle/input/ambulance-dataset/carsdataset',
    labels='inferred',
    label_mode='categorical',
    validation_split=0.2,
    subset='validation', 
    class_names=None,
    color_mode='rgb',
    batch_size=64,
    image_size=(256, 256),
    shuffle=True,
    seed=42                   
)


Found 4749 files belonging to 2 classes.
Using 3800 files for training.
Found 4749 files belonging to 2 classes.
Using 949 files for validation.


In [35]:
base_model=MobileNetV2(
    weights="imagenet",
    input_shape=(256,256,3),
    include_top=False
)
base_model.trainable=False
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255,input_shape=(256,256,3)),
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.2)
])

/tmp/ipykernel_48/2550062916.py:1: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model=MobileNetV2(


In [37]:
inputs=keras.Input(shape=(256,256,3))
x=data_augmentation(inputs)
x=base_model(x, training=False)
x=keras.layers.GlobalAveragePooling2D()(x)
x=keras.layers.Dense(786,activation="relu")(x)
outputs=keras.layers.Dense(2,activation="softmax")(x)
model=keras.Model(inputs,outputs)

In [39]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_ds,validation_data=val_ds,batch_size=64,epochs=3)

Epoch 1/3
60/60 ━━━━━━━━━━━━━━━━━━━━ 14s 140ms/step - accuracy: 0.6823 - loss: 2.0813 - val_accuracy: 0.9958 - val_loss: 0.0199
Epoch 2/3
60/60 ━━━━━━━━━━━━━━━━━━━━ 7s 114ms/step - accuracy: 0.9787 - loss: 0.0589 - val_accuracy: 0.9968 - val_loss: 0.0114
Epoch 3/3
60/60 ━━━━━━━━━━━━━━━━━━━━ 7s 113ms/step - accuracy: 0.9821 - loss: 0.0469 - val_accuracy: 0.9968 - val_loss: 0.0096


In [40]:
base_model.trainable=True
model.compile(optimizer=keras.optimizers.Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_ds,validation_data=val_ds,batch_size=64,epochs=1)

60/60 ━━━━━━━━━━━━━━━━━━━━ 60s 468ms/step - accuracy: 0.8886 - loss: 0.2747 - val_accuracy: 0.9895 - val_loss: 0.0350


In [44]:
model.save("ambulance_detector.keras")

In [42]:
test_ds = tf.keras.utils.image_dataset_from_directory(
    "/kaggle/input/testing",
    labels=None,              
    shuffle=False,            
    image_size=(256, 256),               
)
predictions = model.predict(test_ds)
for image in predictions:
    if image[0]>image[1]:
        print(f"Ambulance with {image[0]*100}%")
    else:
        print(f"Not Ambulance with {image[1]*100}%")

Found 3 files.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Ambulance with 99.853515625%
Not Ambulance with 99.951171875%
Ambulance with 100.0%
